# Unanimous consensus

ANN arquitecture updated.

It needs upgrading to allow fine tuning.

Training with a subset of spectrograms with unanimous consensus.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import regularizers


import matplotlib.pyplot as plt

base_dir = "../../kaggle_data/hms"
# base_dir = "../../data/hms"
# base_dir = "/kaggle/input/hms-harmful-brain-activity-classification"

df_traincsv = pd.read_csv(f'{base_dir}/train.csv')
idxs = np.load("../data/feateng_03_unanimous_idxs.npy")
df = df_traincsv.loc[idxs]


2024-02-01 14:06:22.458786: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-01 14:06:22.480578: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Using provided spectrograms

Loading observations with unanimous consensus.

In [2]:
df

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0,0,0,3,0,0
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0,0,0,3,0,0
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0,0,0,3,0,0
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0,0,0,3,0,0


In [4]:
path_to_data = "../data/00_spectrograms_reduced.npy"
path_to_items = "../data/00_sub_spectrograms_idxs_reduced.npy"

data = np.load(path_to_data)
items_full = np.load(path_to_items)
items_full

array([[3403533082,          0,          0,          3],
       [3403533082,          1,          1,          3],
       [3403533082,          2,          2,          3],
       ...,
       [  72664070,          5,       6516,          0],
       [3155112843,          0,       6496,          5],
       [2448761963,          0,       6496,          1]])

In [5]:
# Saving only items present in the unanimous df.
items = np.array([], dtype=int).reshape(0,4)
for item in items_full:
    if len(df.loc[(df.eeg_id == item[0]) & (df.eeg_sub_id == item[1])]) > 0:
        items = np.concatenate([items, item.reshape(1,4)])
items.shape

(175, 4)

In [7]:
n_total_samples = items.shape[0]

ptrain = 0.8

idx_train = np.array([], dtype=int)
idx_val = np.array([], dtype=int)
for i in np.arange(6):
    idx = np.where(items[:,3] == i)[0]
    idx = np.random.permutation(idx)
    cut = int(ptrain*idx.shape[0])
    idx_train = np.append(idx_train, idx[0:cut])
    idx_val = np.append(idx_val, idx[cut:])

items_train = items[idx_train]
items_val = items[idx_val]
print("Train samples:", len(items_train))
print("Validation samples:", len(items_val))

Train samples: 139
Validation samples: 36


In [8]:
#
# Data generator using numpy and no pandas.
#
# Original spectrograms.
# 50 seconds slice
# 4 channels
#

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, items, data, batch_size=32, n_classes=6, shuffle=True):
        ''' Initialization
        items: [eeg_id, eeg_sub_id, idx of offset, target]
        '''
        self.n_channels = 4
        self.n_freqs = 40
        self.dim = (25, self.n_freqs)

        self.data = data
        self.items = items
        self.batch_size = batch_size
        self.len = items.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))
        y = np.empty((true_size), dtype=int)

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            # Sample is 50 second long, that's 25 rows.
            initial = item[2] + 137
            final = initial + 25
            for c in np.arange(self.n_channels):
                cinitial = c * 100
                cfinal = cinitial + self.n_freqs
                X[i,:,:,c] = self.data[initial:final, cinitial:cfinal]
            # Store class
            y[i] = item[3]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [9]:

def make_model(input_shape, num_classes):
    input_layer = keras.layers.Input(input_shape)

    #max1 = keras.layers.MaxPooling1D(pool_size=2)(input_layer)
    
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=3, strides=1,
                                padding="same", data_format="channels_last",
                                kernel_regularizer=regularizers.l2(0.001),
                                use_bias=True)(input_layer)
    #conv1 = keras.layers.BatchNormalization()(conv1)
    # conv1 = keras.layers.MaxPooling2D(pool_size=8)(conv1)
    conv1 = keras.layers.ReLU()(conv1)
    
    conv2 = keras.layers.Conv2D(filters=32, kernel_size=5, strides=1,
                                padding="same", data_format="channels_last",
                                kernel_regularizer=regularizers.l2(0.001),
                                use_bias=True)(conv1)
    #conv2 = keras.layers.BatchNormalization()(conv2)
    # conv2 = keras.layers.MaxPooling2D(pool_size=8)(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv2D(filters=64, kernel_size=7, strides=1,
                                padding="same", data_format="channels_last",
                                kernel_regularizer=regularizers.l2(0.001),
                                use_bias=True)(conv2)
    #conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)
    conv3 = keras.layers.MaxPooling2D(pool_size=2)(conv3)

    # conv4 = keras.layers.Conv1D(filters=512, kernel_size=3, padding="same")(conv3)
    # conv4 = keras.layers.BatchNormalization()(conv4)
    # conv4 = keras.layers.MaxPooling2D(pool_size=4)(conv4)
    # conv4 = keras.layers.ReLU()(conv4)

    fltn  = keras.layers.Flatten()(conv3) 
    
    dense1 = keras.layers.Dense(256)(fltn)
    dout1 = keras.layers.Dropout(rate=0.4)(dense1)

    # relu2 = keras.layers.Dense(64)(relu1)
    # relu2 = keras.layers.ReLU(64)(relu2)

#     lin = keras.layers.Dense(2)(relu2)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(dout1)
    # output_layer = keras.layers.Dense(num_classes, activation="relu")(dout1)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


In [14]:
# Parameters
params = {
    'batch_size': 32,
    'n_classes': 6,
    'shuffle': True
    }

training_generator = DataGenerator(items_train, data, **params)
validation_generator = DataGenerator(items_val, data, **params)



In [16]:
model = make_model(input_shape=(25,40,4), num_classes=6)
model.compile(optimizer='sgd',
            loss='categorical_crossentropy',
            # metrics=[tf.keras.metrics.CategoricalCrossentropy()])
        )       

model.fit(training_generator, epochs=1, validation_data=validation_generator)


5/5 [==============================] - 1s 98ms/step - loss: nan - val_loss: nan


In [ ]:
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

#
# Test Data generator: for predicting.
#

class test_DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, ids, path_to_test_data, batch_size=32, n_classes=6):
        'Initialization'
        self.n_channels = 4
        self.dim = (25,40)
        self.n_freqs = 40

        self.path = path_to_test_data
        # self.files = os.listdir(path_to_test_data)
        self.ids = ids
        self.indexes = np.arange(len(self.ids))
        # self.columns = self.data.columns[2:]
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(len(self.ids) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        # items_temp = self.items.iloc[indexes]

        # Generate data
        X = self.__data_generation(indexes)

        return X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        # self.indexes = np.arange(self.len)
        # if self.shuffle == True:
        #     np.random.shuffle(self.indexes)
        pass

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((len(indexes), *self.dim, self.n_channels))

        # Generate data
        for i, idx in enumerate(indexes):
            # item = self.items.iloc[idx]
            test_spectrogram = pd.read_parquet(f'{self.path}{self.ids[idx]}.parquet')
            test_spectrogram.replace(np.nan, 0, inplace=True)

            initial = 137
            final = initial + 25
            for c in np.arange(self.n_channels):
                cinitial = c * 100 + 1
                cfinal = cinitial + self.n_freqs
                # X[i,:,:,c] = self.data[initial:final, cinitial:cfinal]
                X[i,:,:,c] = test_spectrogram.iloc[initial:final,cinitial:cfinal].to_numpy(copy=True)

        return X


# Parameters
params = {
    'batch_size': 32,
    'n_classes': 6,
    }


# base_dir = pathlib.Path("/kaggle/input/hms-harmful-brain-activity-classification")

path_to_test_data = f'{base_dir}/test_spectrograms/'
test = pd.read_csv(f'{base_dir}/test.csv')
ids = test['spectrogram_id'].values

test_generator = test_DataGenerator(ids, path_to_test_data, **params)

y_pred = model.predict(test_generator)

sub = pd.DataFrame({'eeg_id':test.eeg_id.values})
sub[TARGETS] = np.round(y_pred,6)
sub.to_csv('submission.csv',index=False)
